In [6]:
from PIL.Image import init
from utils.models import CNNtoRNN, get_device
from utils.dataset import get_dataloader, get_dataset
from utils.train import overfit, train, validate_model
import argparse
import os
import torch

ModuleNotFoundError: No module named 'utils'

In [2]:
device = get_device(1)
dataset = get_dataset("/home/yandex/DLW2021/davidhay/coco/train2017",
                      "/home/yandex/DLW2021/davidhay/coco/annotations/captions_train2017.json",
                      5, True)
data_loader = get_dataloader(dataset, 10, shuffle=False)
embed_size = 512
hidden_size = 512
vocab_size = len(dataset.vocab)


/home/yandex/DLW2021/davidhay/anaconda3/lib/python3.8/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


loading annotations into memory...
Done (t=9.94s)
creating index...
index created!
Processed 1000 images and captions
Processed 2000 images and captions
Processed 3000 images and captions
Processed 4000 images and captions
Processed 5000 images and captions
Processed 6000 images and captions
Processed 7000 images and captions
Processed 8000 images and captions
Processed 9000 images and captions
Processed 10000 images and captions
Processed 11000 images and captions
Processed 12000 images and captions
Processed 13000 images and captions
Processed 14000 images and captions
Processed 15000 images and captions
Processed 16000 images and captions
Processed 17000 images and captions
Processed 18000 images and captions
Processed 19000 images and captions
Processed 20000 images and captions
Processed 21000 images and captions
Processed 22000 images and captions
Processed 23000 images and captions
Processed 24000 images and captions
Processed 25000 images and captions
Processed 26000 images and

Processed 333000 images and captions
Processed 334000 images and captions
Processed 335000 images and captions
Processed 336000 images and captions
Processed 337000 images and captions
Processed 338000 images and captions
Processed 339000 images and captions
Processed 340000 images and captions
Processed 341000 images and captions
Processed 342000 images and captions
Processed 343000 images and captions
Processed 344000 images and captions
Processed 345000 images and captions
Processed 346000 images and captions
Processed 347000 images and captions
Processed 348000 images and captions
Processed 349000 images and captions
Processed 350000 images and captions
Processed 351000 images and captions
Processed 352000 images and captions
Processed 353000 images and captions
Processed 354000 images and captions
Processed 355000 images and captions
Processed 356000 images and captions
Processed 357000 images and captions
Processed 358000 images and captions
Processed 359000 images and captions
P

Loaded vocabulary from vocab.pkl
Using 10218 words


/home/yandex/DLW2021/davidhay/anaconda3/lib/python3.8/site-packages/torch/nn/modules/rnn.py:62: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


<All keys matched successfully>

In [ ]:
model = CNNtoRNN(2048, embed_size, hidden_size, vocab_size)
model.load_state_dict(torch.load("checkpoint.torch")["model_state_dict"])

In [3]:
iterator = iter(data_loader)
img,cap,length = next(iterator)
print(img.shape)

torch.Size([10, 3, 512, 512])


In [4]:
self = model.decoderRNN
print(self)

DecoderRNNV3(
  (embed): Embedding(10218, 512)
  (lstm): LSTM(2560, 512, batch_first=True, dropout=0.2)
  (fc_out): Linear(in_features=512, out_features=10218, bias=True)
)


In [97]:
def caption_features(self, features, vocab, vec_len):
    '''
    Vec_len should be the same as is learning. 
    '''
    model.eval()
    with torch.no_grad():
        features = model.encoderCNN(features)
        assert features.size(
            0) == 1, "Caption features doesn't support batches"
        # features: (B,F) -> (1,1,F)
        # w_embed: (1) -> (1,1,E)
        next_w = torch.tensor(4).to(device)
        next_w = torch.unsqueeze(next_w, 0)
        next_w = torch.unsqueeze(next_w, 0)
        w_embed = self.embed(next_w)
        features = torch.unsqueeze(features, 1)
        hi = torch.zeros((self.num_layers, 1, self.hidden_size)).to(device)
        ci = torch.zeros((self.num_layers, 1, self.hidden_size)).to(device)
        output = []
        words = "<SOS> a bicycle replica with a clock as the front wheel . <EOS>".split(" ")
        words = [1, 4, 96, 3862, 8, 4, 103, 119, 7, 65, 153, 5, 2]
        for i in range(vec_len):
            w = words[i]
            print(f'embedding:{w}')
            print(f'using: {next_w.item()}')
            w = torch.tensor(w).to(device)
            w = torch.unsqueeze(w, 0)
            w = torch.unsqueeze(w, 0)
            w = self.embed(w)

            combined = torch.cat((features, w_embed), dim=2)
            if i == 0:
                lstm_out, (hi, ci) = self.lstm(combined)
            else:
                lstm_out, (hi, ci) = self.lstm(combined, (hi, ci))
            next_w = torch.argmax(self.fc_out(lstm_out), dim=2)
            print(f"GOT:{next_w.item()}")
            output.append(vocab.itos[next_w.item()])
            # lstm_out: (1,1,F)
            # hi, ci: (num_layers, 1, F)
            # next_w: (1,1,vocab_size)
            w_embed = self.embed(next_w)
        return output

In [98]:
prediction = caption_features(self, img[0:1].to(device), vocab=data_loader.dataset.vocab, vec_len=13)
print(f"Prediction:{' '.join(prediction)}")
demo_cap = ' '.join([data_loader.dataset.vocab.itos[idx2.item(
    )] for idx2 in cap[0] if idx2.item() != data_loader.dataset.vocab.stoi["<PAD>"]])
print(f"Original: {demo_cap}") 


embedding:1
using: 4
tensor([4], device='cuda:0')
GOT:4
embedding:4
using: 4
tensor([4], device='cuda:0')
GOT:4
embedding:96
using: 4
tensor([4], device='cuda:0')
GOT:4
embedding:3862
using: 4
tensor([4], device='cuda:0')
GOT:4
embedding:8
using: 4
tensor([4], device='cuda:0')
GOT:4
embedding:4
using: 4
tensor([4], device='cuda:0')
GOT:4
embedding:103
using: 4
tensor([4], device='cuda:0')
GOT:4
embedding:119
using: 4
tensor([4], device='cuda:0')
GOT:4
embedding:7
using: 4
tensor([4], device='cuda:0')
GOT:4
embedding:65
using: 4
tensor([4], device='cuda:0')
GOT:4
embedding:153
using: 4
tensor([4], device='cuda:0')
GOT:4
embedding:5
using: 4
tensor([4], device='cuda:0')
GOT:4
embedding:2
using: 4
tensor([4], device='cuda:0')
GOT:4
Prediction:a a a a a a a a a a a a a
Original: <SOS> a bicycle replica with a clock as the front wheel . <EOS>
